In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np # Often useful, good to import

# --- Step 3.2: Load Data from CSV ---
print("Loading data from 'v_Sales_Performance_Master.csv'...")

# !!! IMPORTANT: Change this to the correct path where you saved your file !!!
file_path = r'C:\Users\jaiku\PycharmProjects\Retail_Sales_Customer_Segementation\SQL_VIEW_Sales_Performance_Master.csv' 

try:
    df_sales = pd.read_csv(file_path)
    print("Data loaded successfully.")
    
    # Convert order_date to datetime objects for calculations
    df_sales['order_date'] = pd.to_datetime(df_sales['order_date'])

except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    print("Please update the 'file_path' variable to the correct location.")
except Exception as e:
    print(f"Error loading data: {e}")

# Check if df_sales was loaded successfully before proceeding
if 'df_sales' in locals():
    # Display the first 5 rows and columns/data types
    print("\n--- DataFrame Head ---")
    print(df_sales.head())
    print("\n--- DataFrame Info ---")
    df_sales.info()

    # --- Step 3.3: Calculate RFM Metrics ---
    print("\n--- Calculating RFM Metrics ---")

    # 1. Calculate Recency
    snapshot_date = df_sales['order_date'].max() + pd.Timedelta(days=1)
    df_recency = df_sales.groupby('customer_id')['order_date'].max().reset_index()
    df_recency['recency'] = (snapshot_date - df_recency['order_date']).dt.days

    # 2. Calculate Frequency
    df_frequency = df_sales.groupby('customer_id')['order_id'].nunique().reset_index()
    df_frequency.columns = ['customer_id', 'frequency']

    # 3. Calculate Monetary
    df_monetary = df_sales.groupby('customer_id')['net_sales'].sum().reset_index()
    df_monetary.columns = ['customer_id', 'monetary']

    # Merge R, F, and M tables into one DataFrame
    df_rfm = df_recency[['customer_id', 'recency']].merge(
        df_frequency, on='customer_id'
    ).merge(
        df_monetary, on='customer_id'
    )

    print("\n--- RFM DataFrame Head ---")
    print(df_rfm.head())

    # --- Step 3.4: Segment Customers with K-Means Clustering ---
    print("\n--- Performing K-Means Clustering ---")

    rfm_for_clustering = df_rfm[['recency', 'frequency', 'monetary']]
    
    scaler = StandardScaler()
    rfm_scaled = scaler.fit_transform(rfm_for_clustering)

    k = 4 # Number of clusters
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(rfm_scaled)

    df_rfm['cluster'] = kmeans.labels_

    print("\n--- RFM DataFrame with Clusters ---")
    print(df_rfm.head())

    # --- Step 3.5: Analyze the Segments ---
    print("\n--- Analyzing Cluster Segments ---")
    cluster_analysis = df_rfm.groupby('cluster')[['recency', 'frequency', 'monetary']].mean().reset_index()
    print(cluster_analysis.sort_values(by='monetary', ascending=False))
    
    # --- Final Step: Save the results ---
    output_csv_path = r'C:\Users\jaiku\PycharmProjects\Retail_Sales_Customer_Segementation\customer_segments.csv'
    df_rfm.to_csv(output_csv_path, index=False)
    print(f"\nCustomer segmentation complete. Results saved to '{output_csv_path}'")

Loading data from 'v_Sales_Performance_Master.csv'...
Data loaded successfully.

--- DataFrame Head ---
   order_id  item_id order_date required_date shipped_date  \
0         1        1 2016-01-01    2016-01-03   2016-01-03   
1         4        4 2016-01-03    2016-01-04   2016-01-05   
2         9        2 2016-01-05    2016-01-08   2016-01-08   
3        12        2 2016-01-06    2016-01-08   2016-01-09   
4        14        1 2016-01-09    2016-01-11   2016-01-12   

   shipment_delay_days  product_id  \
0                  0.0          20   
1                  1.0          16   
2                  0.0          20   
3                  1.0          17   
4                  1.0          18   

                                        product_name      category_name  \
0     Electra Townie Original 7D EQ - Women's - 2016  Cruisers Bicycles   
1               Electra Townie Original 7D EQ - 2016  Cruisers Bicycles   
2     Electra Townie Original 7D EQ - Women's - 2016  Cruisers Bicycl

In [14]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- 1. Load Data ---
# Load the CSV file created in the previous phase
try:
    df_rfm = pd.read_csv('customer_segments.csv')
except FileNotFoundError:
    print("Error: 'customer_segments.csv' not found.")
    print("Please run the previous segmentation script first.")
    exit()

# --- 2. Analyze & Rename Segments ---
# First, let's understand the segments to give them meaningful names
# Group by cluster and find the mean of R, F, and M
df_rfm_agg = df_rfm.groupby('cluster')[['recency', 'frequency', 'monetary']].mean().reset_index()

# Sort by Monetary, Frequency, and Recency to identify the personas
# Sorting by Monetary (High to Low) is a good start
df_rfm_agg = df_rfm_agg.sort_values(by='monetary', ascending=False)

# Create a mapping dictionary for segment names
# This requires you to look at the output of df_rfm_agg and decide
# For example, the cluster with highest monetary & frequency is 'Champions'
# The cluster with highest recency (long time ago) is 'At Risk'
# This is a *sample* mapping, you MUST adjust this based on your data
segment_mapping = {
    df_rfm_agg.iloc[0]['cluster']: 'Champions (High-Value)',
    df_rfm_agg.iloc[1]['cluster']: 'Loyal Customers (Mid-Value)',
    df_rfm_agg.iloc[2]['cluster']: 'New / Potential Customers',
    df_rfm_agg.iloc[3]['cluster']: 'At Risk / Churned'
}

# Apply the new names
df_rfm['segment_name'] = df_rfm['cluster'].map(segment_mapping)
# Also apply to the aggregated table for plotting
df_rfm_agg['segment_name'] = df_rfm_agg['cluster'].map(segment_mapping)

# Ensure segment_name is treated as a category for sorting
df_rfm['segment_name'] = pd.Categorical(df_rfm['segment_name'], categories=[
    'Champions (High-Value)', 
    'Loyal Customers (Mid-Value)', 
    'New / Potential Customers', 
    'At Risk / Churned'
])
df_rfm_agg['segment_name'] = pd.Categorical(df_rfm_agg['segment_name'], categories=[
    'Champions (High-Value)', 
    'Loyal Customers (Mid-Value)', 
    'New / Potential Customers', 
    'At Risk / Churned'
])


print("--- Segment Analysis (Avg. Values) ---")
print(df_rfm_agg.sort_values(by='segment_name'))


# --- 3. Plot 1: 3D Interactive Cluster View (The "Hero" Chart) ---
print("Generating Plot 1: 3D RFM Cluster Plot")
fig_3d = px.scatter_3d(
    df_rfm,
    x='recency',
    y='frequency',
    z='monetary',
    color='segment_name',
    color_discrete_map={
        'Champions (High-Value)': '#107C10', # Green
        'Loyal Customers (Mid-Value)': '#0078D4', # Blue
        'New / Potential Customers': '#FFB900', # Yellow/Gold
        'At Risk / Churned': '#D83B01' # Red/Orange
    },
    title='3D Interactive RFM Customer Segments',
    hover_data=['customer_id']
)

fig_3d.update_layout(
    margin=dict(l=0, r=0, b=0, t=40),
    legend_title_text='Customer Segment'
)
fig_3d.show()


# --- 4. Plot 2: Segment Characteristics (Bar Charts) ---
print("Generating Plot 2: Segment Characteristics")
# We will create 3 subplots (one for R, one for F, one for M)
fig_bars = make_subplots(
    rows=1, cols=3,
    subplot_titles=('Average Recency (Days)', 'Average Frequency (Orders)', 'Average Monetary Value ($)')
)

# Bar plot for Recency
fig_bars.add_trace(go.Bar(
    x=df_rfm_agg['segment_name'],
    y=df_rfm_agg['recency'],
    name='Recency',
    marker_color=px.colors.sequential.Blues_r
), row=1, col=1)

# Bar plot for Frequency
fig_bars.add_trace(go.Bar(
    x=df_rfm_agg['segment_name'],
    y=df_rfm_agg['frequency'],
    name='Frequency',
    marker_color=px.colors.sequential.Greens_r
), row=1, col=2)

# Bar plot for Monetary
fig_bars.add_trace(go.Bar(
    x=df_rfm_agg['segment_name'],
    y=df_rfm_agg['monetary'],
    name='Monetary',
    marker_color=px.colors.sequential.Oranges_r
), row=1, col=3)

fig_bars.update_layout(
    title_text='Customer Segment Characteristics',
    showlegend=False
)
fig_bars.show()


# --- 5. Plot 3: Segment Size & Value (Treemap) ---
print("Generating Plot 3: Segment Size & Value Treemap")
# To get the count of customers in each segment
df_rfm_counts = df_rfm.groupby('segment_name').agg(
    customer_count=('customer_id', 'count'),
    total_value=('monetary', 'sum')
).reset_index()

fig_tree = px.treemap(
    df_rfm_counts,
    path=[px.Constant("All Customers"), 'segment_name'], # Create levels
    values='customer_count',
    color='total_value',
    color_continuous_scale='Greens',
    title='Customer Segment Size (by Count) & Total Value (by Color)',
    hover_data={'segment_name': False, 'customer_count': ':.', 'total_value': ':$,.2f'}
)

fig_tree.update_traces(
    texttemplate="<b>%{label}</b><br>%{value} Customers<br>%{percentParent:.1%} of Total",
    textposition="middle center"
)
fig_tree.update_layout(margin=dict(l=0, r=0, b=0, t=30))
fig_tree.show()

--- Segment Analysis (Avg. Values) ---
   cluster     recency  frequency     monetary                 segment_name
1        1  658.875969   1.000000  5143.911199       Champions (High-Value)
2        2  247.207692   2.300000  3086.778779  Loyal Customers (Mid-Value)
3        3  483.205502   1.000000   967.061232    New / Potential Customers
0        0  887.663732   1.001761   960.432788            At Risk / Churned
Generating Plot 1: 3D RFM Cluster Plot


Generating Plot 2: Segment Characteristics


Generating Plot 3: Segment Size & Value Treemap


C:\Users\jaiku\AppData\Local\Temp\ipykernel_4304\2344381323.py:127: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\jaiku\anaconda3\Lib\site-packages\plotly\express\_core.py:1727: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

